## RSS and activity of regulons in control vs disease

In [ ]:
import scanpy as sc
import loompy as lp
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import random
import numpy as np
from math import sqrt, ceil, floor
import seaborn as sns
import glob
import os
import sys
from datetime import datetime
from scipy.stats import ttest_ind
from scipy.spatial.distance import jensenshannon
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import StandardScaler
import warnings

sc.set_figure_params(figsize=(4,4))
sc.set_figure_params(dpi=200)
sc.settings.n_jobs = 1 #nCores/CPUs for scanpy

# for white background of figures (only for docs rendering)
%config InlineBackend.print_figure_kwargs={'facecolor' : "w"}
%config InlineBackend.figure_format='retina'

#hpc figures
%matplotlib inline

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" # print multiple outputs per code cell (not just last)

In [ ]:
# Helper functions
def regulon_specificity_scores(auc_mtx, cell_type_series):
    """
    Calculates the Regulon Specificty Scores (RSS). [doi: 10.1016/j.celrep.2018.10.045]

    :param auc_mtx: The dataframe with the AUC values for all cells and regulons (n_cells x n_regulons).
    :param cell_type_series: A pandas Series object with cell identifiers as index and cell type labels as values.
    :return: A pandas dataframe with the RSS values (cell type x regulon).
    """

    cell_types = list(cell_type_series.unique())
    n_types = len(cell_types)
    regulons = list(auc_mtx.columns)
    n_regulons = len(regulons)
    rss_values = np.empty(shape=(n_types, n_regulons), dtype=np.float32)

    def rss(aucs, labels):
        # jensenshannon function provides distance which is the sqrt of the JS divergence.
        return 1.0 - jensenshannon(aucs / aucs.sum(), labels / labels.sum())

    for cidx, regulon_name in enumerate(regulons):
        for ridx, cell_type in enumerate(cell_types):
            rss_values[ridx, cidx] = rss(
                auc_mtx[regulon_name], (cell_type_series == cell_type).astype(int)
            )

    return pd.DataFrame(data=rss_values, index=cell_types, columns=regulons)

def plot_rss(rss, cell_type, top_n=5, max_n=None, ax=None):
    if ax is None:
        _, ax = plt.subplots(1, 1, figsize=(4, 4))
    if max_n is None:
        max_n = rss.shape[1]
    data = rss.T[cell_type].sort_values(ascending=False)[0:max_n]
    ax.plot(np.arange(len(data)), data, ".")
    ax.set_ylim([floor(data.min() * 100.0) / 100.0, ceil(data.max() * 100.0) / 100.0])
    ax.set_ylabel("RSS")
    ax.set_xlabel("Regulon")
    ax.set_title(cell_type)
    ax.set_xticklabels([])

    font = {
        "color": "red",
        "weight": "normal",
        "size": 6,
    }

    for idx, (regulon_name, rss_val) in enumerate(
        zip(data[0:top_n].index, data[0:top_n].values)
    ):
        ax.plot([idx, idx], [rss_val, rss_val], "r.")
        ax.text(
            idx + (max_n / 25),
            rss_val,
            regulon_name,
            fontdict=font,
            horizontalalignment="left",
            verticalalignment="center",
        )
# Create category representations in metacells

def annotate_category(obs, metacells, column, n_cells, add_prefix) :
    cat = list(obs[column].unique())

    vals = {}

    for i in cat: 
            vals[i] = []       

    for i in metacells.uns['neighbor_dict'].keys(): 
        for j in cat :
            vals[j].append(sum(obs[column][metacells.uns['neighbor_dict'][i]] == j) / n_cells)
            
    try :
        sum(metacells.obs.index == list(metacells.uns['neighbor_dict'].keys())) == len(metacells.uns['neighbor_dict']) 
        
        for i in vals :
            metacells.obs[f'{add_prefix}_{i}'] = vals[i]
    except : 
        print("Index of neighbor dictionary and obs do not match")
def celltype_frac(df, cells, col_name):
    val_counts = df.loc[cells,col_name].value_counts()
    return val_counts.values[0] / val_counts.values.sum()
# select PCs with elbow locator

def select_pcs_knee(adata, plot = False) : 
    x = np.array(adata.uns['pca']['variance_ratio'])
    y = np.array(list(range(1,len(x) +1)))
    
    kneedle = KneeLocator(x, y, curve="convex", direction="decreasing")
    if plot :
        sc.pl.pca_variance_ratio(adata, n_pcs=len(x), show=False)
        plt.axvline(x=kneedle.knee_y)
        plt.show()
    
    return kneedle.knee_y + 1
seed = 250
def set_seed(seed=int): # Set seed
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    print('Seed set to', seed)

set_seed(seed)

In [3]:
work_dir = "/scratch/gent/vo/000/gvo00027/projects/CBIGR/21HPP_GRN_neuroinfl/singlecell/SCENIC_rebuttal_VZZ/"

### 1. Calculate RSS scores separately for disease and control

#### AD

In [ ]:
!ls -l /scratch/gent/vo/000/gvo00027/projects/CBIGR/21JDS_singlecell_GBM/SCENIC/SCENIC_vsn/scripts/HPU_SCENIC_thesis/counts_AD1/scenic/counts_AD1

In [ ]:
# Import activities

loom_ad = lp.connect("/scratch/gent/vo/000/gvo00027/projects/CBIGR/21JDS_singlecell_GBM/SCENIC/SCENIC_vsn/scripts/HPU_SCENIC_thesis/counts_AD1/scenic/counts_AD1/SCENIC_output.loom" , mode='r', validate=False)
auc_ad = pd.DataFrame(loom_ad.ca.MotifRegulonsAUC, index=loom_ad.ca.CellID)
auc_ad.head()

In [ ]:
# Metadata

metadata_ad = pd.read_csv("/scratch/gent/vo/000/gvo00027/projects/CBIGR/21HPP_GRN_neuroinfl/singlecell/snRNA_metadta.csv", index_col=0)
metadata_ad.head()

In [ ]:
# Unify cell names 

auc_ad.index = [i.replace(".", "-") for i in auc_ad.index.values]
auc_ad.head()

In [ ]:
# Subset to cell present in AUC matrix

metadata_ad = metadata_ad.loc[auc_ad.index, :]
metadata_ad.shape

In [ ]:
# Subset for disease and control 

auc_ad_dis = auc_ad.loc[metadata_ad["Diagnosis"] == "AD",].copy()
auc_ad_ctrl = auc_ad.loc[metadata_ad["Diagnosis"] == "Control",].copy()

auc_ad_dis.shape
auc_ad_ctrl.shape

In [ ]:
# Calculate RSS

rss_ad_dis = regulon_specificity_scores(auc_mtx=auc_ad_dis, cell_type_series=metadata_ad.loc[metadata_ad["Diagnosis"] == "AD", "celltype"])
rss_ad_dis

In [13]:
crc_tfs = ["IKZF1", "IRF8", "NFATC2", "MAF", "RUNX1", "TAL1", "SPI1", "MAFF", "STAT3", "MYC", "FOS", "JUNB", "CEBPD", "PLSCR1"]

In [ ]:
rss_ranks_ad_dis = rss_ad_dis.copy()

for ct in rss_ranks_ad_dis.index :
    rss_ranks_ad_dis.loc[ct] =  np.argsort(np.argsort(rss_ranks_ad_dis.loc[ct]))

rss_ad_dis
rss_ranks_ad_dis

In [ ]:
rss_ranks_ad_dis.max(axis=1)
rss_ranks_ad_dis.min(axis=1)

In [ ]:
rss_ranks_ad_dis

In [ ]:
rss_ranks_ad_dis.columns = [i.split("_")[0] for i in rss_ranks_ad_dis.columns.values]
rss_ranks_ad_dis = rss_ranks_ad_dis.loc[:,rss_ranks_ad_dis.columns.isin(crc_tfs)]
rss_ranks_ad_dis

In [ ]:
for i in metadata_ad.celltype.unique() :
    plot_rss(rss_ad_dis, cell_type=i)

In [ ]:
rss_ad_ctrl = regulon_specificity_scores(auc_mtx=auc_ad_ctrl, cell_type_series=metadata_ad.loc[metadata_ad["Diagnosis"] == "Control", "celltype"])
rss_ad_ctrl

In [ ]:
rss_ranks_ad_ctrl = rss_ad_ctrl.copy()

for ct in rss_ranks_ad_ctrl.index :
    rss_ranks_ad_ctrl.loc[ct] =  np.argsort(np.argsort(rss_ranks_ad_ctrl.loc[ct]))

rss_ad_ctrl
rss_ranks_ad_ctrl

In [ ]:
rss_ranks_ad_ctrl.max(axis=1)
rss_ranks_ad_ctrl.min(axis=1)

In [ ]:
rss_ranks_ad_ctrl

In [ ]:
rss_ranks_ad_ctrl.columns = [i.split("_")[0] for i in rss_ranks_ad_ctrl.columns.values]
rss_ranks_ad_ctrl = rss_ranks_ad_ctrl.loc[:,rss_ranks_ad_ctrl.columns.isin(crc_tfs)]
rss_ranks_ad_ctrl

In [ ]:
for i in metadata_ad.celltype.unique() :
    plot_rss(rss_ad_ctrl, cell_type=i)

In [ ]:
rss_ranks_ad_ctrl.columns = [f"{i}_ctrl" for i in rss_ranks_ad_ctrl.columns.values]
rss_ranks_ad_ctrl

#### Create plots

##### RSS rank change

In [ ]:
rss_ranks_ad_dis.columns = [f"{i}_dis" for i in rss_ranks_ad_dis.columns.values]
rss_ranks_ad_dis

In [ ]:
rss_ranks_ad_comp = pd.concat([rss_ranks_ad_dis, rss_ranks_ad_ctrl], axis=1)
# rss_ranks_ad_comp['celltype'] = rss_ranks_ad_comp.index
rss_ranks_ad_comp

In [ ]:
# Melt the DataFrame

for ct in rss_ranks_ad_comp.index : 

    df_plot = pd.DataFrame(rss_ranks_ad_comp.loc[ct], columns=[ct]).T
    df_plot = df_plot.melt(var_name="Variable_Condition", value_name="Value")
    df_plot[['Variable', 'Condition']] = df_plot['Variable_Condition'].str.rsplit('_', n=1, expand=True)

    # Plot
    plt.figure(figsize=(2, 4))
    sns.lineplot(
        data=df_plot,
        x="Condition",
        y="Value",
        hue="Variable",
        estimator=None,  # Show all data points without aggregation
        markers=True,
        style="Variable",
        dashes=False,
        alpha=0.7
    )
    plt.title(f"Change in RSS ranks in AD - {ct}")

    # Reverse axis
    # plt.gca().invert_yaxis()
    plt.gca().invert_xaxis()

    plt.ylim(1,359)

    # Move legend to the right
    plt.legend(title="Regulon", bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.grid(False)
    plt.savefig(f"scenic_postpr_heatmaps/RSSranks_CRCs_AD_{ct}.png", bbox_inches="tight")
    plt.savefig(f"scenic_postpr_heatmaps/RSSranks_CRCs_AD_{ct}.svg", bbox_inches="tight", format="svg")
    plt.savefig(f"scenic_postpr_heatmaps/RSSranks_CRCs_AD_{ct}.pdf", bbox_inches="tight", format="pdf")
    plt.show()


Different approach

In [17]:
os.mkdir("RSS_CRC_Tables")

In [ ]:
for ct in metadata_ad.celltype.unique() :
    
    print("Calculating for", ct)

    rss = regulon_specificity_scores(auc_mtx=auc_ad.loc[metadata_ad["celltype"] == ct, :], cell_type_series=metadata_ad.loc[metadata_ad["celltype"] == ct, "Diagnosis"])
    for cd in rss.index :
        rss.loc[cd] =  np.argsort(np.argsort(rss.loc[cd]))

    rss.columns = [i.split("_")[0] for i in rss.columns.values]
    rss = rss.loc[:, rss.columns.isin(crc_tfs)]
    rss.T.to_csv(f"RSS_CRC_Tables/RSS_CRC_AD_{ct}.csv")

    rss['condition'] = rss.index
    rss_melt = pd.melt(rss, id_vars='condition')

    # Plot
    plt.figure(figsize=(2, 4))
    sns.lineplot(
        data=rss_melt,
        x="condition",
        y="value",
        hue="variable",
        estimator=None,  # Show all data points without aggregation
        markers=True,
        style="variable",
        dashes=False,
        alpha=0.7
    )
    plt.title(f"Change in RSS ranks in AD - {ct}")

    # Reverse axis
    # plt.gca().invert_yaxis()
    plt.gca().invert_xaxis()

    plt.ylim(1,359)

    # Move legend to the right
    plt.legend(title="Regulon", bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.grid(False)
    # plt.savefig(f"scenic_postpr_heatmaps/RSSranks_setup2_CRCs_AD_{ct}.png", bbox_inches="tight")
    # plt.savefig(f"scenic_postpr_heatmaps/RSSranks_setup2_CRCs_AD_{ct}.svg", bbox_inches="tight", format="svg")
    # plt.savefig(f"scenic_postpr_heatmaps/RSSranks_setup2_CRCs_AD_{ct}.pdf", bbox_inches="tight", format="pdf")
    plt.show()


##### Dotheatmap

Disease

In [ ]:
for ct in rss_ad_dis.index :
    rss_ad_dis.loc[ct] =  np.argsort(np.argsort(rss_ad_dis.loc[ct]))

rss_ad_dis

In [ ]:
rss_ad_dis.max(axis=1)
rss_ad_dis.min(axis=1)

In [ ]:
rss_ad_dis["celltype"] = rss_ad_dis.index
rss_ad_dis

In [ ]:
# Order celltypes

rss_ad_dis.sort_index(inplace=True)
rss_ad_dis

In [ ]:
rss_long = pd.melt(rss_ad_dis, id_vars="celltype")
rss_long.columns = ["celltype", "Regulon", "RSS rank"]
rss_long

In [ ]:
rss_long["Activity"] = 0
rss_long

In [425]:
for reg in auc_ad_dis.columns : 

    for cs in rss_long.celltype.unique() : 

        mean_act = np.mean(auc_ad_dis.loc[metadata_ad.loc[metadata_ad["Diagnosis"] == "AD",]["celltype"] == cs,reg])
        rss_long.loc[(rss_long["Regulon"] == reg) & (rss_long["celltype"] == cs), "Activity"] = mean_act

In [ ]:
rss_long

In [ ]:
# Import expression data

ad_mat_dis = sc.read_text("/scratch/gent/vo/000/gvo00027/projects/CBIGR/21HPP_GRN_neuroinfl/singlecell/counts_AD1.txt.gz")

# dimensions are in R format
genes = ad_mat_dis.obs.index
cells = ad_mat_dis.var.index

cells = [i.replace(".", "-") for i in cells]

adata_ad = sc.AnnData(ad_mat_dis.T)

ad_mat_dis.obs.index = cells
ad_mat_dis.var.index = genes

ad_mat_dis

In [ ]:
rss_long["Counts"] = 0
rss_long

In [ ]:
exp_ad_dis = pd.DataFrame(ad_mat_dis.X.T, index=cells, columns=genes)
exp_ad_dis

In [429]:
for reg in rss_long.Regulon.unique() : 
    tf_name = reg.split("_")[0]
    for cs in rss_long.celltype.unique() : 
        if tf_name in genes :
            avg_exp = np.mean(exp_ad_dis.loc[ metadata_ad.loc[(metadata_ad["Diagnosis"] == "AD") & (metadata_ad["celltype"] == cs)].index ][tf_name])
        else :
            # quick "fix" but check with Hanne
            avg_exp = 0
        rss_long.loc[(rss_long["Regulon"] == reg) & (rss_long["celltype"] == cs), "Counts"] = avg_exp

In [ ]:
rss_long

In [431]:
# Select 5 most active TFs in each cell state

auc_top10 = []
for cs in rss_long.celltype.unique() : 
    tmp = rss_long[rss_long["celltype"] == cs].sort_values(by="Activity", ascending=False)
    auc_top10.extend(list(tmp["Regulon"][:10].values))

In [ ]:
len(auc_top10)
len(set(auc_top10))

In [433]:
# Make list unique while preserving order
auc_top10 = list(dict.fromkeys(auc_top10))

In [ ]:
rss_long= rss_long.loc[rss_long["Regulon"].isin(auc_top10),:].copy()
rss_long

In [435]:
# Get unique categories for both axes
unique_tf = rss_long['Regulon'].unique()
unique_cs = rss_long['celltype'].unique()

# Calculate the center positions of each tile
tile_centers = [(i, j) for i in range(len(unique_tf)) for j in range(len(unique_cs))]

In [ ]:
pivot_df = rss_long.pivot(index='celltype', columns='Regulon', values='Activity')
pivot_df

In [ ]:
pivot_df = pivot_df[auc_top10]
pivot_df

In [ ]:
pivot_df.columns = [reg.split("_")[0] for reg in pivot_df.columns]
pivot_df.head()

In [ ]:
import matplotlib as mpl

plt.figure(figsize=(15, 3), dpi=300)
sns.heatmap(pivot_df, cmap='magma_r')

cmap = mpl.colormaps.get_cmap('cividis')
for i, (tf, cs) in enumerate(tile_centers):
    size = rss_long.loc[(rss_long['Regulon'] == auc_top10[tf]) & (rss_long['celltype'] == unique_cs[cs]),'RSS rank'].values[0]
    color_value = rss_long.loc[(rss_long['Regulon'] == auc_top10[tf]) & (rss_long['celltype'] == unique_cs[cs]),'Counts'].values[0]

    color = cmap(color_value)
    plt.scatter(tf + 0.5, cs + 0.5, s=size/2 , color=color)

plt.xlabel('TF')
plt.xticks([i+.5 for i in range(pivot_df.shape[1])], list(pivot_df.columns), fontsize=16, rotation=90)
plt.ylabel('Celltype')
plt.yticks([i+.5 for i in range(pivot_df.shape[0])], list(pivot_df.index), fontsize=16)
plt.grid(False)
plt.savefig(f"scenic_postpr_heatmaps/Dotheatmap_AD_disease_TFsbyAUC.png", bbox_inches="tight")
plt.savefig(f"scenic_postpr_heatmaps/Dotheatmap_AD_disease_TFsbyAUC.svg", bbox_inches="tight", format="svg")
plt.savefig(f"scenic_postpr_heatmaps/Dotheatmap_AD_disease_TFsbyAUC.pdf", bbox_inches="tight", format="pdf")
plt.show();

In [ ]:
rss_long["Counts"].max()

In [ ]:
fig,ax = plt.subplots(figsize=(1,3), dpi=300)

#adding colorbar
norm = mpl.colors.Normalize(vmin=0, vmax=3)
sm = plt.cm.ScalarMappable(cmap=cmap,norm=norm)
sm.set_array([])
cbar = fig.colorbar(sm,ax)
for t in cbar.ax.get_yticklabels():
    t.set_fontsize(15)
    
plt.title("TF expression\n", fontsize=20)
plt.grid(False)
plt.savefig(f"scenic_postpr_heatmaps/Dotheatmap_AD_disease_TFsbyAUC_TFexp_Legend.png", bbox_inches="tight")
plt.savefig(f"scenic_postpr_heatmaps/Dotheatmap_AD_disease_TFsbyAUC_TFexp_Legend.svg", bbox_inches="tight", format="svg")
plt.savefig(f"scenic_postpr_heatmaps/Dotheatmap_AD_disease_TFsbyAUC_TFexp_Legend.pdf", bbox_inches="tight", format="pdf")
plt.show();

In [ ]:
numbers = [1, 2, 3, 4]
val = np.round( np.quantile(rss_long["RSS rank"].values, [.25, .5, .75, 1]), 0)
sizes= val /2

# Create a figure without any content
fig = plt.figure(figsize=(1, 3), dpi=300)

# Create an empty axis
ax = fig.add_subplot(111)

# Plot dots with invisible markers for creating legend
for num, size, v in zip(numbers, sizes, val):
    ax.scatter([], [], s=size, label=f'{v}', color='black')

# Add legend
ax.legend(title='RSS rank\n', loc='center')

# Hide axis
ax.axis('off')
plt.savefig(f"scenic_postpr_heatmaps/Dotheatmap_AD_disease_TFsbyAUC_RSSrank_Legend.png", bbox_inches="tight")
plt.savefig(f"scenic_postpr_heatmaps/Dotheatmap_AD_disease_TFsbyAUC_RSSrank_Legend.svg", bbox_inches="tight", format="svg")
plt.savefig(f"scenic_postpr_heatmaps/Dotheatmap_AD_disease_TFsbyAUC_RSSrank_Legend.pdf", bbox_inches="tight", format="pdf")
plt.show();

Control - TODO

In [ ]:
for ct in rss_ad_ctrl.index :
    rss_ad_ctrl.loc[ct] =  np.argsort(np.argsort(rss_ad_ctrl.loc[ct]))

rss_ad_ctrl

In [ ]:
rss_ad_ctrl.max(axis=1)
rss_ad_ctrl.min(axis=1)

In [ ]:
rss_ad_ctrl.sort_index(inplace=True)
rss_ad_ctrl

In [ ]:
rss_ad_ctrl["celltype"] = rss_ad_ctrl.index
rss_ad_ctrl

In [ ]:
rss_long = pd.melt(rss_ad_ctrl, id_vars="celltype")
rss_long.columns = ["celltype", "Regulon", "RSS rank"]
rss_long

In [ ]:
rss_long["Activity"] = 0
rss_long

In [346]:
for reg in auc_ad_ctrl.columns : 

    for cs in rss_long.celltype.unique() : 

        mean_act = np.mean(auc_ad_ctrl.loc[metadata_ad.loc[metadata_ad["Diagnosis"] == "Control","celltype"] == cs,reg])
        rss_long.loc[(rss_long["Regulon"] == reg) & (rss_long["celltype"] == cs), "Activity"] = mean_act

In [ ]:
rss_long["Counts"] = 0
rss_long

In [348]:
for reg in rss_long.Regulon.unique() : 
    tf_name = reg.split("_")[0]
    for cs in rss_long.celltype.unique() : 
        if tf_name in genes :
            avg_exp = np.mean(exp_ad_dis.loc[ metadata_ad.loc[(metadata_ad["Diagnosis"] == "AD") & (metadata_ad["celltype"] == cs)].index ][tf_name])
        else :
            # quick "fix" but check with Hanne
            avg_exp = 0
        rss_long.loc[(rss_long["Regulon"] == reg) & (rss_long["celltype"] == cs), "Counts"] = avg_exp

In [ ]:
rss_long

In [350]:
# Select 10 most active TFs in each cell state

auc_top10 = []
for cs in rss_long.celltype.unique() : 
    tmp = rss_long[rss_long["celltype"] == cs].sort_values(by="Activity", ascending=False)
    auc_top10.extend(list(tmp["Regulon"][:10].values))

In [ ]:
len(auc_top10)
len(set(auc_top10))

In [352]:
# Make list unique while preserving order
auc_top10 = list(dict.fromkeys(auc_top10))

In [ ]:
rss_long= rss_long.loc[rss_long["Regulon"].isin(auc_top10),:].copy()
rss_long

In [354]:
# Get unique categories for both axes
unique_tf = rss_long['Regulon'].unique()
unique_cs = rss_long['celltype'].unique()

# Calculate the center positions of each tile
tile_centers = [(i, j) for i in range(len(unique_tf)) for j in range(len(unique_cs))]

In [ ]:
pivot_df = rss_long.pivot(index='celltype', columns='Regulon', values='Activity')
pivot_df

In [ ]:
pivot_df = pivot_df[auc_top10]
pivot_df

In [ ]:
pivot_df.columns = [reg.split("_")[0] for reg in pivot_df.columns]
pivot_df.head()

In [ ]:
import matplotlib as mpl

plt.figure(figsize=(15, 3), dpi=300)
sns.heatmap(pivot_df, cmap='magma_r')

cmap = mpl.colormaps.get_cmap('cividis')
for i, (tf, cs) in enumerate(tile_centers):
    size = rss_long.loc[(rss_long['Regulon'] == auc_top10[tf]) & (rss_long['celltype'] == unique_cs[cs]),'RSS rank'].values[0]
    color_value = rss_long.loc[(rss_long['Regulon'] == auc_top10[tf]) & (rss_long['celltype'] == unique_cs[cs]),'Counts'].values[0]

    color = cmap(color_value)
    plt.scatter(tf + 0.5, cs + 0.5, s=size/2 , color=color)

plt.xlabel('TF')
plt.xticks([i+.5 for i in range(pivot_df.shape[1])], list(pivot_df.columns), fontsize=16, rotation=90)
plt.ylabel('Celltype')
plt.yticks([i+.5 for i in range(pivot_df.shape[0])], list(pivot_df.index), fontsize=16)
plt.grid(False)
plt.savefig(f"scenic_postpr_heatmaps/Dotheatmap_AD_control_TFsbyAUC.png", bbox_inches="tight")
plt.savefig(f"scenic_postpr_heatmaps/Dotheatmap_AD_control_TFsbyAUC.svg", bbox_inches="tight", format="svg")
plt.savefig(f"scenic_postpr_heatmaps/Dotheatmap_AD_control_TFsbyAUC.pdf", bbox_inches="tight", format="pdf")
plt.show();

In [ ]:
rss_long["Counts"].max()

In [ ]:
fig,ax = plt.subplots(figsize=(1,3), dpi=300)

#adding colorbar
norm = mpl.colors.Normalize(vmin=0, vmax=3)
sm = plt.cm.ScalarMappable(cmap=cmap,norm=norm)
sm.set_array([])
cbar = fig.colorbar(sm,ax)
for t in cbar.ax.get_yticklabels():
    t.set_fontsize(15)
    
plt.title("TF expression\n", fontsize=20)
plt.grid(False)
plt.savefig(f"scenic_postpr_heatmaps/Dotheatmap_AD_control_TFsbyAUC_TFexp_Legend.png", bbox_inches="tight")
plt.savefig(f"scenic_postpr_heatmaps/Dotheatmap_AD_control_TFsbyAUC_TFexp_Legend.svg", bbox_inches="tight", format="svg")
plt.savefig(f"scenic_postpr_heatmaps/Dotheatmap_AD_control_TFsbyAUC_TFexp_Legend.pdf", bbox_inches="tight", format="pdf")
plt.show();

In [ ]:
numbers = [1, 2, 3, 4]
val = np.round( np.quantile(rss_long["RSS rank"].values, [.25, .5, .75, 1]), 0)
sizes= val /2

# Create a figure without any content
fig = plt.figure(figsize=(1, 3), dpi=300)

# Create an empty axis
ax = fig.add_subplot(111)

# Plot dots with invisible markers for creating legend
for num, size, v in zip(numbers, sizes, val):
    ax.scatter([], [], s=size, label=f'{v}', color='black')

# Add legend
ax.legend(title='RSS rank\n', loc='center')

# Hide axis
ax.axis('off')
plt.savefig(f"scenic_postpr_heatmaps/Dotheatmap_AD_control_TFsbyAUC_RSSrank_Legend.png", bbox_inches="tight")
plt.savefig(f"scenic_postpr_heatmaps/Dotheatmap_AD_control_TFsbyAUC_RSSrank_Legend.svg", bbox_inches="tight", format="svg")
plt.savefig(f"scenic_postpr_heatmaps/Dotheatmap_AD_control_TFsbyAUC_RSSrank_Legend.pdf", bbox_inches="tight", format="pdf")
plt.show();

### MDD

In [ ]:
!ls -l /scratch/gent/vo/000/gvo00027/projects/CBIGR/21JDS_singlecell_GBM/SCENIC/SCENIC_vsn/scripts/HPU_SCENIC_thesis/counts_MDD/scenic/counts_MDD

In [ ]:
# Import activities

loom_mdd = lp.connect("/scratch/gent/vo/000/gvo00027/projects/CBIGR/21JDS_singlecell_GBM/SCENIC/SCENIC_vsn/scripts/HPU_SCENIC_thesis/counts_MDD/scenic/counts_MDD/SCENIC_output.loom" , mode='r', validate=False)
auc_mdd = pd.DataFrame(loom_mdd.ca.MotifRegulonsAUC, index=loom_mdd.ca.CellID)
auc_mdd.head()

In [ ]:
# Metadata

metadata_mdd = pd.read_csv("/scratch/gent/vo/000/gvo00027/projects/CBIGR/21HPP_GRN_neuroinfl/singlecell/Metadata_MDD.txt", index_col=0, sep="\t")
metadata_mdd.head()

In [ ]:
# Unify cell names 

auc_mdd.index = [i.replace("Micro.Macro", "Micro/Macro") for i in auc_mdd.index.values]
auc_mdd.head()

In [ ]:
# Subset to cell present in AUC matrix

metadata_mdd = metadata_mdd.loc[auc_mdd.index, :]
metadata_mdd.shape

In [ ]:
# Subset for disease and control 

auc_mdd_dis = auc_mdd.loc[metadata_mdd["Diagnosis"] == "Suicide",].copy()
auc_mdd_ctrl = auc_mdd.loc[metadata_mdd["Diagnosis"] == "Control",].copy()

auc_mdd_dis.shape
auc_mdd_ctrl.shape

In [ ]:
# Calculate RSS

rss_mdd_dis = regulon_specificity_scores(auc_mtx=auc_mdd_dis, cell_type_series=metadata_mdd.loc[metadata_mdd["Diagnosis"] == "Suicide", "Celltype"])
rss_mdd_dis

In [ ]:
rss_ranks_mdd_dis = rss_mdd_dis.copy()

for ct in rss_ranks_mdd_dis.index :
    rss_ranks_mdd_dis.loc[ct] =  np.argsort(np.argsort(rss_ranks_mdd_dis.loc[ct]))

rss_mdd_dis
rss_ranks_mdd_dis

In [ ]:
rss_ranks_mdd_dis.max(axis=1)
rss_ranks_mdd_dis.min(axis=1)

In [ ]:
rss_ranks_mdd_dis.columns = [i.split("_")[0] for i in rss_ranks_mdd_dis.columns.values]
rss_ranks_mdd_dis = rss_ranks_mdd_dis.loc[:,rss_ranks_mdd_dis.columns.isin(crc_tfs)]
rss_ranks_mdd_dis

In [ ]:
for i in metadata_mdd.Celltype.unique() :
    plot_rss(rss_mdd_dis, cell_type=i)

In [ ]:
rss_mdd_ctrl = regulon_specificity_scores(auc_mtx=auc_mdd_ctrl, cell_type_series=metadata_mdd.loc[metadata_mdd["Diagnosis"] == "Control", "Celltype"])
rss_mdd_ctrl

In [ ]:
rss_ranks_mdd_ctrl = rss_mdd_ctrl.copy()

for ct in rss_ranks_mdd_ctrl.index :
    rss_ranks_mdd_ctrl.loc[ct] =  np.argsort(np.argsort(rss_ranks_mdd_ctrl.loc[ct]))

rss_mdd_ctrl
rss_ranks_mdd_ctrl

In [ ]:
rss_ranks_mdd_ctrl.max(axis=1)
rss_ranks_mdd_ctrl.min(axis=1)

In [ ]:
rss_ranks_mdd_ctrl.columns = [i.split("_")[0] for i in rss_ranks_mdd_ctrl.columns.values]
rss_ranks_mdd_ctrl = rss_ranks_mdd_ctrl.loc[:,rss_ranks_mdd_ctrl.columns.isin(crc_tfs)]
rss_ranks_mdd_ctrl

In [ ]:
for i in metadata_mdd.Celltype.unique() :
    plot_rss(rss_mdd_ctrl, cell_type=i)

In [ ]:
rss_ranks_mdd_ctrl.columns = [f"{i}_ctrl" for i in rss_ranks_mdd_ctrl.columns.values]
rss_ranks_mdd_ctrl

#### Create plots

##### RSS rank change

In [ ]:
rss_ranks_mdd_dis.columns = [f"{i}_dis" for i in rss_ranks_mdd_dis.columns.values]
rss_ranks_mdd_dis

In [ ]:
rss_ranks_mdd_comp = pd.concat([rss_ranks_mdd_dis, rss_ranks_mdd_ctrl], axis=1)
rss_ranks_mdd_comp

In [ ]:
# Plot change in RSS rank 

for ct in rss_ranks_mdd_comp.index : 

    df_plot = pd.DataFrame(rss_ranks_mdd_comp.loc[ct], columns=[ct]).T
    df_plot = df_plot.melt(var_name="Variable_Condition", value_name="Value")
    df_plot[['Variable', 'Condition']] = df_plot['Variable_Condition'].str.rsplit('_', n=1, expand=True)

    # Plot
    plt.figure(figsize=(2, 4))
    sns.lineplot(
        data=df_plot,
        x="Condition",
        y="Value",
        hue="Variable",
        estimator=None,  # Show all data points without aggregation
        markers=True,
        style="Variable",
        dashes=False,
        alpha=0.7
    )
    plt.title(f"Change in RSS ranks in MDD - {ct}")

    # Reverse axis
    # plt.gca().invert_yaxis()
    plt.gca().invert_xaxis()


    plt.ylim(1,265)
    # Move legend to the right
    plt.legend(title="Regulon", bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.grid(False)
    plt.savefig(f"scenic_postpr_heatmaps/RSSranks_CRCs_MDD_{ct}.png", bbox_inches="tight")
    plt.savefig(f"scenic_postpr_heatmaps/RSSranks_CRCs_MDD_{ct}.svg", bbox_inches="tight", format="svg")
    plt.savefig(f"scenic_postpr_heatmaps/RSSranks_CRCs_MDD_{ct}.pdf", bbox_inches="tight", format="pdf")
    plt.show()


Different approach

In [ ]:
for ct in metadata_mdd.Celltype.unique() :
    
    print("Calculating for", ct)

    rss = regulon_specificity_scores(auc_mtx=auc_mdd.loc[metadata_mdd["Celltype"] == ct, :], cell_type_series=metadata_mdd.loc[metadata_mdd["Celltype"] == ct, "Diagnosis"])
    for cd in rss.index :
        rss.loc[cd] =  np.argsort(np.argsort(rss.loc[cd]))

    rss.columns = [i.split("_")[0] for i in rss.columns.values]
    rss = rss.loc[:, rss.columns.isin(crc_tfs)]
    rss.T.to_csv(f"RSS_CRC_Tables/RSS_CRC_MDD_{ct}.csv")

    rss['condition'] = rss.index
    rss_melt = pd.melt(rss, id_vars='condition')

    # Plot
    plt.figure(figsize=(2, 4))
    sns.lineplot(
        data=rss_melt,
        x="condition",
        y="value",
        hue="variable",
        estimator=None,  # Show all data points without aggregation
        markers=True,
        style="variable",
        dashes=False,
        alpha=0.7
    )
    plt.title(f"Change in RSS ranks in MDD - {ct}")

    # Reverse axis
    # plt.gca().invert_yaxis()
    # plt.gca().invert_xaxis()

    plt.ylim(1,359)

    # Move legend to the right
    plt.legend(title="Regulon", bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.grid(False)
    # plt.savefig(f"scenic_postpr_heatmaps/RSSranks_setup2_CRCs_MDD_{ct}.png", bbox_inches="tight")
    # plt.savefig(f"scenic_postpr_heatmaps/RSSranks_setup2_CRCs_MDD_{ct}.svg", bbox_inches="tight", format="svg")
    # plt.savefig(f"scenic_postpr_heatmaps/RSSranks_setup2_CRCs_MDD_{ct}.pdf", bbox_inches="tight", format="pdf")
    plt.show()

In [ ]:
rss

##### Dotheatmap

Disease

In [ ]:
for ct in rss_mdd_dis.index :
    rss_mdd_dis.loc[ct] =  np.argsort(np.argsort(rss_mdd_dis.loc[ct]))

rss_mdd_dis

In [ ]:
rss_mdd_dis.max(axis=1)
rss_mdd_dis.min(axis=1)

In [ ]:
rss_mdd_dis["celltype"] = rss_mdd_dis.index
rss_mdd_dis

In [ ]:
# Oder celltypes

rss_mdd_dis.sort_index(inplace=True)
rss_mdd_dis

In [ ]:
rss_long = pd.melt(rss_mdd_dis, id_vars="celltype")
rss_long.columns = ["celltype", "Regulon", "RSS rank"]
rss_long

In [ ]:
rss_long["Activity"] = 0
rss_long

In [377]:
for reg in auc_mdd_dis.columns : 

    for cs in rss_long.celltype.unique() : 

        mean_act = np.mean(auc_mdd_dis.loc[metadata_mdd.loc[metadata_mdd["Diagnosis"] == "Suicide","Celltype"] == cs,reg])
        rss_long.loc[(rss_long["Regulon"] == reg) & (rss_long["celltype"] == cs), "Activity"] = mean_act

In [ ]:
rss_long

In [ ]:
# Import expression data

mdd_mat_dis = sc.read_text("/scratch/gent/vo/000/gvo00027/projects/CBIGR/21HPP_GRN_neuroinfl/singlecell/counts_MDD.txt.gz")

# dimensions are in R format
genes = mdd_mat_dis.obs.index
cells = mdd_mat_dis.var.index

cells = [i.replace("Micro.Macro", "Micro/Macro") for i in cells]

adata_mdd = sc.AnnData(mdd_mat_dis.T)

mdd_mat_dis.obs.index = cells
mdd_mat_dis.var.index = genes

mdd_mat_dis

In [ ]:
rss_long["Counts"] = 0
rss_long

In [ ]:
exp_mdd_dis = pd.DataFrame(mdd_mat_dis.X.T, index=cells, columns=genes)
exp_mdd_dis

In [380]:
for reg in rss_long.Regulon.unique() : 
    tf_name = reg.split("_")[0]
    for cs in rss_long.celltype.unique() : 
        if tf_name in genes :
            avg_exp = np.mean(exp_mdd_dis.loc[ metadata_mdd.loc[(metadata_mdd["Diagnosis"] == "Suicide") & (metadata_mdd["Celltype"] == cs)].index ][tf_name])
        else :
            
            # quick "fix" but check with Hanne
            avg_exp = 0
        rss_long.loc[(rss_long["Regulon"] == reg) & (rss_long["celltype"] == cs), "Counts"] = avg_exp

In [ ]:
rss_long

In [382]:
# Select 10 most active TFs in each cell state

auc_top10 = []
for cs in rss_long.celltype.unique() : 
    tmp = rss_long[rss_long["celltype"] == cs].sort_values(by="Activity", ascending=False)
    auc_top10.extend(list(tmp["Regulon"][:10].values))

In [ ]:
len(auc_top10)
len(set(auc_top10))

In [384]:
# Make list unique while preserving order
auc_top10 = list(dict.fromkeys(auc_top10))

In [ ]:
rss_long= rss_long.loc[rss_long["Regulon"].isin(auc_top10),:].copy()
rss_long

In [386]:
# Get unique categories for both axes
unique_tf = rss_long['Regulon'].unique()
unique_cs = rss_long['celltype'].unique()

# Calculate the center positions of each tile
tile_centers = [(i, j) for i in range(len(unique_tf)) for j in range(len(unique_cs))]

In [ ]:
pivot_df = rss_long.pivot(index='celltype', columns='Regulon', values='Activity')
pivot_df

In [ ]:
pivot_df = pivot_df[auc_top10]
pivot_df

In [ ]:
pivot_df.columns = [reg.split("_")[0] for reg in pivot_df.columns]
pivot_df.head()

In [391]:
# Cap expression at 2

rss_long.loc[rss_long["Counts"] > 2, "Counts"] = 2

In [ ]:
import matplotlib as mpl

plt.figure(figsize=(15, 3), dpi=300)
sns.heatmap(pivot_df, cmap='magma_r')

cmap = mpl.colormaps.get_cmap('cividis')
for i, (tf, cs) in enumerate(tile_centers):
    size = rss_long.loc[(rss_long['Regulon'] == auc_top10[tf]) & (rss_long['celltype'] == unique_cs[cs]),'RSS rank'].values[0]
    color_value = rss_long.loc[(rss_long['Regulon'] == auc_top10[tf]) & (rss_long['celltype'] == unique_cs[cs]),'Counts'].values[0]

    color = cmap(color_value)
    plt.scatter(tf + 0.5, cs + 0.5, s=size/2 , color=color)

plt.xlabel('TF')
plt.xticks([i+.5 for i in range(pivot_df.shape[1])], list(pivot_df.columns), fontsize=16, rotation=90)
plt.ylabel('Celltype')
plt.yticks([i+.5 for i in range(pivot_df.shape[0])], list(pivot_df.index), fontsize=16)
plt.grid(False)
plt.savefig(f"scenic_postpr_heatmaps/Dotheatmap_MDD_disease_TFsbyAUC.png", bbox_inches="tight")
plt.savefig(f"scenic_postpr_heatmaps/Dotheatmap_MDD_disease_TFsbyAUC.svg", bbox_inches="tight", format="svg")
plt.savefig(f"scenic_postpr_heatmaps/Dotheatmap_MDD_disease_TFsbyAUC.pdf", bbox_inches="tight", format="pdf")
plt.show();

In [ ]:
rss_long["Counts"].max()

In [ ]:
fig,ax = plt.subplots(figsize=(1,3), dpi=300)

#adding colorbar
norm = mpl.colors.Normalize(vmin=0, vmax=2)
sm = plt.cm.ScalarMappable(cmap=cmap,norm=norm)
sm.set_array([])
cbar = fig.colorbar(sm,ax)
for t in cbar.ax.get_yticklabels():
    t.set_fontsize(15)
    
plt.title("TF expression\n", fontsize=20)
plt.grid(False)
plt.savefig(f"scenic_postpr_heatmaps/Dotheatmap_MDD_disease_TFsbyAUC_TFexp_Legend.png", bbox_inches="tight")
plt.savefig(f"scenic_postpr_heatmaps/Dotheatmap_MDD_disease_TFsbyAUC_TFexp_Legend.svg", bbox_inches="tight", format="svg")
plt.savefig(f"scenic_postpr_heatmaps/Dotheatmap_MDD_disease_TFsbyAUC_TFexp_Legend.pdf", bbox_inches="tight", format="pdf")
plt.show();

In [ ]:
numbers = [1, 2, 3, 4]
val = np.round( np.quantile(rss_long["RSS rank"].values, [.25, .5, .75, 1]), 0)
sizes= val /2

# Create a figure without any content
fig = plt.figure(figsize=(1, 3), dpi=300)

# Create an empty axis
ax = fig.add_subplot(111)

# Plot dots with invisible markers for creating legend
for num, size, v in zip(numbers, sizes, val):
    ax.scatter([], [], s=size, label=f'{v}', color='black')

# Add legend
ax.legend(title='RSS rank\n', loc='center')

# Hide axis
ax.axis('off')
plt.savefig(f"scenic_postpr_heatmaps/Dotheatmap_MDD_disease_TFsbyAUC_RSSrank_Legend.png", bbox_inches="tight")
plt.savefig(f"scenic_postpr_heatmaps/Dotheatmap_MDD_disease_TFsbyAUC_RSSrank_Legend.svg", bbox_inches="tight", format="svg")
plt.savefig(f"scenic_postpr_heatmaps/Dotheatmap_MDD_disease_TFsbyAUC_RSSrank_Legend.pdf", bbox_inches="tight", format="pdf")
plt.show();

Control

In [ ]:
for ct in rss_mdd_ctrl.index :
    rss_mdd_ctrl.loc[ct] =  np.argsort(np.argsort(rss_mdd_ctrl.loc[ct]))

rss_mdd_ctrl

In [ ]:
rss_mdd_ctrl.max(axis=1)
rss_mdd_ctrl.min(axis=1)

In [ ]:
rss_mdd_ctrl["celltype"] = rss_mdd_ctrl.index
rss_mdd_ctrl

In [ ]:
# Oder celltypes

rss_mdd_ctrl.sort_index(inplace=True)
rss_mdd_ctrl

In [ ]:
rss_long = pd.melt(rss_mdd_ctrl, id_vars="celltype")
rss_long.columns = ["celltype", "Regulon", "RSS rank"]
rss_long

In [ ]:
rss_long["Activity"] = 0
rss_long

In [401]:
for reg in auc_mdd_ctrl.columns : 

    for cs in rss_long.celltype.unique() : 

        mean_act = np.mean(auc_mdd_ctrl.loc[metadata_mdd.loc[metadata_mdd["Diagnosis"] == "Control","Celltype"] == cs,reg])
        rss_long.loc[(rss_long["Regulon"] == reg) & (rss_long["celltype"] == cs), "Activity"] = mean_act

In [ ]:
rss_long

In [ ]:
rss_long["Counts"] = 0
rss_long

In [404]:
for reg in rss_long.Regulon.unique() : 
    tf_name = reg.split("_")[0]
    for cs in rss_long.celltype.unique() : 
        if tf_name in genes :
            avg_exp = np.mean(exp_mdd_dis.loc[ metadata_mdd.loc[(metadata_mdd["Diagnosis"] == "Control") & (metadata_mdd["Celltype"] == cs)].index ][tf_name])
        else :
            
            # quick "fix" but check with Hanne
            avg_exp = 0
        rss_long.loc[(rss_long["Regulon"] == reg) & (rss_long["celltype"] == cs), "Counts"] = avg_exp

In [ ]:
rss_long

In [406]:
# Select 10 most active TFs in each cell state

auc_top10 = []
for cs in rss_long.celltype.unique() : 
    tmp = rss_long[rss_long["celltype"] == cs].sort_values(by="Activity", ascending=False)
    auc_top10.extend(list(tmp["Regulon"][:10].values))

In [ ]:
len(auc_top10)
len(set(auc_top10))

In [408]:
# Make list unique while preserving order
auc_top10 = list(dict.fromkeys(auc_top10))

In [ ]:
rss_long= rss_long.loc[rss_long["Regulon"].isin(auc_top10),:].copy()
rss_long

In [410]:
# Get unique categories for both axes
unique_tf = rss_long['Regulon'].unique()
unique_cs = rss_long['celltype'].unique()

# Calculate the center positions of each tile
tile_centers = [(i, j) for i in range(len(unique_tf)) for j in range(len(unique_cs))]

In [ ]:
pivot_df = rss_long.pivot(index='celltype', columns='Regulon', values='Activity')
pivot_df

In [ ]:
pivot_df = pivot_df[auc_top10]
pivot_df

In [ ]:
pivot_df.columns = [reg.split("_")[0] for reg in pivot_df.columns]
pivot_df.head()

In [415]:
# Cap expression at 2

rss_long.loc[rss_long["Counts"] > 2, "Counts"] = 2

In [ ]:
import matplotlib as mpl

plt.figure(figsize=(15, 3), dpi=300)
sns.heatmap(pivot_df, cmap='magma_r')

cmap = mpl.colormaps.get_cmap('cividis')
for i, (tf, cs) in enumerate(tile_centers):
    size = rss_long.loc[(rss_long['Regulon'] == auc_top10[tf]) & (rss_long['celltype'] == unique_cs[cs]),'RSS rank'].values[0]
    color_value = rss_long.loc[(rss_long['Regulon'] == auc_top10[tf]) & (rss_long['celltype'] == unique_cs[cs]),'Counts'].values[0]

    color = cmap(color_value)
    plt.scatter(tf + 0.5, cs + 0.5, s=size/2 , color=color)

plt.xlabel('TF')
plt.xticks([i+.5 for i in range(pivot_df.shape[1])], list(pivot_df.columns), fontsize=14, rotation=90)
plt.ylabel('Celltype')
plt.yticks([i+.5 for i in range(pivot_df.shape[0])], list(pivot_df.index), fontsize=16)
plt.grid(False)
plt.savefig(f"scenic_postpr_heatmaps/Dotheatmap_MDD_control_TFsbyAUC.png", bbox_inches="tight")
plt.savefig(f"scenic_postpr_heatmaps/Dotheatmap_MDD_control_TFsbyAUC.svg", bbox_inches="tight", format="svg")
plt.savefig(f"scenic_postpr_heatmaps/Dotheatmap_MDD_control_TFsbyAUC.pdf", bbox_inches="tight", format="pdf")
plt.show();

In [ ]:
fig,ax = plt.subplots(figsize=(1,3), dpi=300)

#adding colorbar
norm = mpl.colors.Normalize(vmin=0, vmax=2)
sm = plt.cm.ScalarMappable(cmap=cmap,norm=norm)
sm.set_array([])
cbar = fig.colorbar(sm,ax)
for t in cbar.ax.get_yticklabels():
    t.set_fontsize(15)
    
plt.title("TF expression\n", fontsize=20)
plt.grid(False)
plt.savefig(f"scenic_postpr_heatmaps/Dotheatmap_MDD_control_TFsbyAUC_TFexp_Legend.png", bbox_inches="tight")
plt.savefig(f"scenic_postpr_heatmaps/Dotheatmap_MDD_control_TFsbyAUC_TFexp_Legend.svg", bbox_inches="tight", format="svg")
plt.savefig(f"scenic_postpr_heatmaps/Dotheatmap_MDD_control_TFsbyAUC_TFexp_Legend.pdf", bbox_inches="tight", format="pdf")
plt.show();

In [ ]:
numbers = [1, 2, 3, 4]
val = np.round( np.quantile(rss_long["RSS rank"].values, [.25, .5, .75, 1]), 0)
sizes= val /2

# Create a figure without any content
fig = plt.figure(figsize=(1, 3), dpi=300)

# Create an empty axis
ax = fig.add_subplot(111)

# Plot dots with invisible markers for creating legend
for num, size, v in zip(numbers, sizes, val):
    ax.scatter([], [], s=size, label=f'{v}', color='black')

# Add legend
ax.legend(title='RSS rank\n', loc='center')

# Hide axis
ax.axis('off')
plt.savefig(f"scenic_postpr_heatmaps/Dotheatmap_MDD_control_TFsbyAUC_RSSrank_Legend.png", bbox_inches="tight")
plt.savefig(f"scenic_postpr_heatmaps/Dotheatmap_MDD_control_TFsbyAUC_RSSrank_Legend.svg", bbox_inches="tight", format="svg")
plt.savefig(f"scenic_postpr_heatmaps/Dotheatmap_MDD_control_TFsbyAUC_RSSrank_Legend.pdf", bbox_inches="tight", format="pdf")
plt.show();

### MDD female

In [ ]:
!ls -l /scratch/gent/vo/000/gvo00027/projects/CBIGR/21HPP_GRN_neuroinfl/singlecell/SCENIC_rebuttal_VZZ/MDD_female/scenic/MDD_female

In [ ]:
# Import activities

loom_mdd = lp.connect("/scratch/gent/vo/000/gvo00027/projects/CBIGR/21HPP_GRN_neuroinfl/singlecell/SCENIC_rebuttal_VZZ/MDD_female/scenic/MDD_female/SCENIC_output_MDD_female.loom" , mode='r', validate=False)
auc_mdd = pd.DataFrame(loom_mdd.ca.MotifRegulonsAUC, index=loom_mdd.ca.CellID)
auc_mdd.head()

In [ ]:
!pwd

In [136]:
auc_mdd.to_csv("/scratch/gent/vo/000/gvo00027/projects/CBIGR/21HPP_GRN_neuroinfl/singlecell/SCENIC_rebuttal_VZZ/MDD_female/MDD_female_AUCmat.csv")

In [ ]:
# Metadata

metadata_mdd = pd.read_csv("/scratch/gent/vo/000/gvo00027/projects/CBIGR/21HPP_GRN_neuroinfl/singlecell/MDD_M_F_metadata.csv", index_col=0)
metadata_mdd.head()

In [ ]:
# Unify cell names 

auc_mdd.index = [i.replace("Micro.Macro", "Micro/Macro") for i in auc_mdd.index.values]
auc_mdd.head()

In [ ]:
auc_mdd.shape

In [ ]:
# Subset to cell present in AUC matrix

metadata_mdd = metadata_mdd.loc[auc_mdd.index, :]
metadata_mdd.shape

Different approach

In [ ]:
for ct in metadata_mdd.Celltype.unique() :
    
    print("Calculating for", ct)

    rss = regulon_specificity_scores(auc_mtx=auc_mdd.loc[metadata_mdd["Celltype"] == ct, :], cell_type_series=metadata_mdd.loc[metadata_mdd["Celltype"] == ct, "Diagnosis"])
    for cd in rss.index :
        rss.loc[cd] =  np.argsort(np.argsort(rss.loc[cd]))

    rss.columns = [i.split("_")[0] for i in rss.columns.values]
    rss = rss.loc[:, rss.columns.isin(crc_tfs)]
    rss.T.to_csv(f"RSS_CRC_Tables/RSS_CRC_MDD_F_{ct}.csv")

    rss['condition'] = rss.index
    rss_melt = pd.melt(rss, id_vars='condition')

    # Plot
    plt.figure(figsize=(2, 4))
    sns.lineplot(
        data=rss_melt,
        x="condition",
        y="value",
        hue="variable",
        estimator=None,  # Show all data points without aggregation
        markers=True,
        style="variable",
        dashes=False,
        alpha=0.7
    )
    plt.title(f"Change in RSS ranks in MDD - {ct}")

    # Reverse axis
    # plt.gca().invert_yaxis()
    # plt.gca().invert_xaxis()

    plt.ylim(1,377)

    # Move legend to the right
    plt.legend(title="Regulon", bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.grid(False)
    plt.savefig(f"scenic_postpr_heatmaps/RSSranks_setup2_CRCs_MDD_F_{ct}.png", bbox_inches="tight")
    plt.savefig(f"scenic_postpr_heatmaps/RSSranks_setup2_CRCs_MDD_F_{ct}.svg", bbox_inches="tight", format="svg")
    plt.savefig(f"scenic_postpr_heatmaps/RSSranks_setup2_CRCs_MDD_F_{ct}.pdf", bbox_inches="tight", format="pdf")
    plt.show()

In [ ]:
rss

##### Dotheatmap

Aggregate

In [ ]:
rss_mdd = regulon_specificity_scores(auc_mtx=auc_mdd, cell_type_series=metadata_mdd["Celltype"])
rss_mdd

In [ ]:
for ct in rss_mdd.index :
    rss_mdd.loc[ct] =  np.argsort(np.argsort(rss_mdd.loc[ct]))

rss_mdd

In [ ]:
rss_mdd.max(axis=1)
rss_mdd.min(axis=1)

In [ ]:
rss_mdd["celltype"] = rss_mdd.index
rss_mdd

In [ ]:
# Order celltypes

rss_mdd.sort_index(inplace=True)
rss_mdd

In [ ]:
rss_long = pd.melt(rss_mdd, id_vars="celltype")
rss_long.columns = ["celltype", "Regulon", "RSS rank"]
rss_long

In [ ]:
rss_long["Activity"] = 0
rss_long

In [16]:
for reg in auc_mdd.columns : 

    for cs in rss_long.celltype.unique() : 

        mean_act = np.mean(auc_mdd.loc[metadata_mdd["Celltype"] == cs,reg])
        rss_long.loc[(rss_long["Regulon"] == reg) & (rss_long["celltype"] == cs), "Activity"] = mean_act

In [ ]:
rss_long

In [18]:
# Import expression data

mdd_mat = sp.sparse.load_npz("/scratch/gent/vo/000/gvo00027/projects/CBIGR/21HPP_GRN_neuroinfl/singlecell/Female_samples_MDD_pyimport/Seurat_MDD_female_pyimport/Seurat_MDD_female.npz")

#genes
genes = []
with open("/scratch/gent/vo/000/gvo00027/projects/CBIGR/21HPP_GRN_neuroinfl/singlecell/Female_samples_MDD_pyimport/Seurat_MDD_female_pyimport/Seurat_MDD_female_featurenames.txt", "r") as f :
    for g in f :
        genes.append(g.strip())

cells = []
with open("/scratch/gent/vo/000/gvo00027/projects/CBIGR/21HPP_GRN_neuroinfl/singlecell/Female_samples_MDD_pyimport/Seurat_MDD_female_pyimport/Seurat_MDD_female_cellnames.txt", "r") as f :
    for c in f :
        cells.append(c.strip())

# cells = [i.replace("Micro.Macro", "Micro/Macro") for i in cells]


In [ ]:
exp_mdd = pd.DataFrame(mdd_mat.T.toarray(), index=cells, columns=genes)
exp_mdd


In [ ]:
rss_long["Counts"] = 0
rss_long

In [21]:
for reg in rss_long.Regulon.unique() : 
    tf_name = reg.split("_")[0]
    for cs in rss_long.celltype.unique() : 
        if tf_name in genes :
            avg_exp = np.mean(exp_mdd.loc[ metadata_mdd.loc[metadata_mdd["Celltype"] == cs,:].index ][tf_name])
        else :
            
            # quick "fix" but check with Hanne
            avg_exp = 0
        rss_long.loc[(rss_long["Regulon"] == reg) & (rss_long["celltype"] == cs), "Counts"] = avg_exp

In [ ]:
rss_long

In [33]:
# Select 10 most active TFs in each cell state

auc_top10 = []
for cs in rss_long.celltype.unique() : 
    tmp = rss_long[rss_long["celltype"] == cs].sort_values(by="Activity", ascending=False)
    auc_top10.extend(list(tmp["Regulon"][:10].values))

In [ ]:
len(auc_top10)
len(set(auc_top10))

In [ ]:
# Make list unique while preserving order
auc_top10 = list(dict.fromkeys(auc_top10))
len(auc_top10)

In [ ]:
# Scale TF expression per cell type

for ct in rss_long.celltype.unique() :
    rss_long.loc[rss_long["celltype"] == ct, "Counts"] = rss_long.loc[rss_long["celltype"] == ct, "Counts"] / rss_long.loc[rss_long["celltype"] == ct, "Counts"].max()

rss_long


In [ ]:
# Scale activity per cell type

for ct in rss_long.celltype.unique() :
    rss_long.loc[rss_long["celltype"] == ct, "Activity"] = rss_long.loc[rss_long["celltype"] == ct, "Activity"] / rss_long.loc[rss_long["celltype"] == ct, "Activity"].max()

rss_long

In [ ]:
rss_long["Counts"].max()
rss_long["Activity"].max()

In [ ]:
rss_long= rss_long.loc[rss_long["Regulon"].isin(auc_top10),:].copy()
rss_long

In [37]:
# Get unique categories for both axes
unique_tf = rss_long['Regulon'].unique()
unique_cs = rss_long['celltype'].unique()

# Calculate the center positions of each tile
tile_centers = [(i, j) for i in range(len(unique_tf)) for j in range(len(unique_cs))]

In [ ]:
pivot_df = rss_long.pivot(index='celltype', columns='Regulon', values='Activity')
pivot_df

In [ ]:
pivot_df = pivot_df[auc_top10]
pivot_df

In [ ]:
pivot_df.columns = [reg.split("-mot")[0] for reg in pivot_df.columns]
pivot_df.head()

In [57]:
import matplotlib.font_manager as fm

font_path = '/user/gent/448/vsc44828/arial_font/ARIAL.TTF'
arial_font = fm.FontProperties(fname=font_path)

In [48]:
pivot_df.index = [
    "Astrocytes", 
    "Endothelial cells", 
    "Excitatory neurons", 
    "Inhibitory neurons", 
    "Microglia", 
    "Mix", 
    "OPCs", 
    "Oligodendrocytes"
]

In [ ]:
import matplotlib as mpl

plt.figure(figsize=(15, 3), dpi=300)

# Apply the font to tick labels
# for label in ax.get_xticklabels() + ax.get_yticklabels():
#     label.set_fontproperties(arial_font)

sns.heatmap(pivot_df, cmap='magma_r')

cmap = mpl.colormaps.get_cmap('cividis')
for i, (tf, cs) in enumerate(tile_centers):
    size = rss_long.loc[(rss_long['Regulon'] == auc_top10[tf]) & (rss_long['celltype'] == unique_cs[cs]),'RSS rank'].values[0]
    color_value = rss_long.loc[(rss_long['Regulon'] == auc_top10[tf]) & (rss_long['celltype'] == unique_cs[cs]),'Counts'].values[0]

    color = cmap(color_value)
    plt.scatter(tf + 0.5, cs + 0.5, s=size/4 , color=color)

plt.xlabel('TF',fontproperties=arial_font)
plt.xticks([i+.5 for i in range(pivot_df.shape[1])], list(pivot_df.columns), fontsize=1, rotation=90, fontproperties=arial_font)
plt.ylabel('Celltype',fontproperties=arial_font)
plt.yticks([i+.5 for i in range(pivot_df.shape[0])], list(pivot_df.index), fontsize=12,fontproperties=arial_font)
plt.grid(False)
plt.savefig(f"scenic_postpr_heatmaps/Dotheatmap_MDD_F_total_TFsbyAUC.png", bbox_inches="tight")
plt.savefig(f"scenic_postpr_heatmaps/Dotheatmap_MDD_F_total_TFsbyAUC.svg", bbox_inches="tight", format="svg")
plt.savefig(f"scenic_postpr_heatmaps/Dotheatmap_MDD_F_total_TFsbyAUC.pdf", bbox_inches="tight", format="pdf")
plt.show();

In [ ]:
numbers = [1, 2, 3, 4, 5]
val = np.round( np.quantile(rss_long["RSS rank"].values, [0, .25, .5, .75, 1]), 0)
sizes= val /4

# Create a figure without any content
fig = plt.figure(figsize=(1, 3), dpi=300)

# Create an empty axis
ax = fig.add_subplot(111)

# Plot dots with invisible markers for creating legend
for num, size, v in zip(numbers, sizes, val):
    ax.scatter([], [], s=size, label=f'{v}', color='black')

# Add legend
ax.legend(title='RSS rank\n', loc='center')

# Hide axis
ax.axis('off')
plt.savefig(f"scenic_postpr_heatmaps/Dotheatmap_MDD_F_total_TFsbyAUC_RSSrank_Legend.png", bbox_inches="tight")
plt.savefig(f"scenic_postpr_heatmaps/Dotheatmap_MDD_F_total_TFsbyAUC_RSSrank_Legend.svg", bbox_inches="tight", format="svg")
plt.savefig(f"scenic_postpr_heatmaps/Dotheatmap_MDD_F_total_TFsbyAUC_RSSrank_Legend.pdf", bbox_inches="tight", format="pdf")
plt.show();

Disease

In [ ]:
# Subset for disease and control 

auc_mdd_dis = auc_mdd.loc[metadata_mdd["Diagnosis"] == "Suicide",].copy()
auc_mdd_ctrl = auc_mdd.loc[metadata_mdd["Diagnosis"] == "Control",].copy()

auc_mdd_dis.shape
auc_mdd_ctrl.shape

In [ ]:
# Calculate RSS

rss_mdd_dis = regulon_specificity_scores(auc_mtx=auc_mdd_dis, cell_type_series=metadata_mdd.loc[metadata_mdd["Diagnosis"] == "Suicide", "Celltype"])
rss_mdd_dis

In [ ]:
for ct in rss_mdd_dis.index :
    rss_mdd_dis.loc[ct] =  np.argsort(np.argsort(rss_mdd_dis.loc[ct]))

rss_mdd_dis

In [ ]:
rss_mdd_dis.max(axis=1)
rss_mdd_dis.min(axis=1)

In [ ]:
rss_mdd_dis["celltype"] = rss_mdd_dis.index
rss_mdd_dis

In [ ]:
# Oder celltypes

rss_mdd_dis.sort_index(inplace=True)
rss_mdd_dis

In [ ]:
rss_long = pd.melt(rss_mdd_dis, id_vars="celltype")
rss_long.columns = ["celltype", "Regulon", "RSS rank"]
rss_long

In [ ]:
rss_long["Activity"] = 0
rss_long

In [76]:
for reg in auc_mdd_dis.columns : 

    for cs in rss_long.celltype.unique() : 

        mean_act = np.mean(auc_mdd_dis.loc[metadata_mdd.loc[metadata_mdd["Diagnosis"] == "Suicide","Celltype"] == cs,reg])
        rss_long.loc[(rss_long["Regulon"] == reg) & (rss_long["celltype"] == cs), "Activity"] = mean_act

In [ ]:
rss_long

In [ ]:
rss_long["Counts"] = 0
rss_long

In [83]:
for reg in rss_long.Regulon.unique() : 
    tf_name = reg.split("_")[0]
    for cs in rss_long.celltype.unique() : 
        if tf_name in genes :
            avg_exp = np.mean(exp_mdd.loc[ metadata_mdd.loc[(metadata_mdd["Diagnosis"] == "Suicide") & (metadata_mdd["Celltype"] == cs)].index ][tf_name])
        else :
            
            # quick "fix" but check with Hanne
            avg_exp = 0
        rss_long.loc[(rss_long["Regulon"] == reg) & (rss_long["celltype"] == cs), "Counts"] = avg_exp

In [ ]:
rss_long

In [85]:
# Select 10 most active TFs in each cell state

auc_top10 = []
for cs in rss_long.celltype.unique() : 
    tmp = rss_long[rss_long["celltype"] == cs].sort_values(by="Activity", ascending=False)
    auc_top10.extend(list(tmp["Regulon"][:10].values))

In [ ]:
len(auc_top10)
len(set(auc_top10))

In [87]:
# Make list unique while preserving order
auc_top10 = list(dict.fromkeys(auc_top10))

In [ ]:
len(auc_top10)

In [ ]:
rss_long= rss_long.loc[rss_long["Regulon"].isin(auc_top10),:].copy()
rss_long

In [90]:
# Get unique categories for both axes
unique_tf = rss_long['Regulon'].unique()
unique_cs = rss_long['celltype'].unique()

# Calculate the center positions of each tile
tile_centers = [(i, j) for i in range(len(unique_tf)) for j in range(len(unique_cs))]

In [ ]:
pivot_df = rss_long.pivot(index='celltype', columns='Regulon', values='Activity')
pivot_df

In [ ]:
pivot_df = pivot_df[auc_top10]
pivot_df

In [ ]:
pivot_df.columns = [reg.split("_")[0] for reg in pivot_df.columns]
pivot_df.head()

In [ ]:
rss_long["Counts"].max()

In [95]:
# Cap expression at 2

rss_long.loc[rss_long["Counts"] > 2, "Counts"] = 2

In [ ]:
import matplotlib as mpl

plt.figure(figsize=(15, 3), dpi=300)
sns.heatmap(pivot_df, cmap='magma_r')

cmap = mpl.colormaps.get_cmap('cividis')
for i, (tf, cs) in enumerate(tile_centers):
    size = rss_long.loc[(rss_long['Regulon'] == auc_top10[tf]) & (rss_long['celltype'] == unique_cs[cs]),'RSS rank'].values[0]
    color_value = rss_long.loc[(rss_long['Regulon'] == auc_top10[tf]) & (rss_long['celltype'] == unique_cs[cs]),'Counts'].values[0]

    color = cmap(color_value)
    plt.scatter(tf + 0.5, cs + 0.5, s=size/3 , color=color)

plt.xlabel('TF')
plt.xticks([i+.5 for i in range(pivot_df.shape[1])], list(pivot_df.columns), fontsize=12, rotation=90)
plt.ylabel('Celltype')
plt.yticks([i+.5 for i in range(pivot_df.shape[0])], list(pivot_df.index), fontsize=12)
plt.grid(False)
plt.savefig(f"scenic_postpr_heatmaps/Dotheatmap_MDD_F_disease_TFsbyAUC.png", bbox_inches="tight")
plt.savefig(f"scenic_postpr_heatmaps/Dotheatmap_MDD_F_disease_TFsbyAUC.svg", bbox_inches="tight", format="svg")
plt.savefig(f"scenic_postpr_heatmaps/Dotheatmap_MDD_F_disease_TFsbyAUC.pdf", bbox_inches="tight", format="pdf")
plt.show();

In [ ]:
rss_long["Counts"].max()

In [ ]:
fig,ax = plt.subplots(figsize=(1,3), dpi=300)

#adding colorbar
norm = mpl.colors.Normalize(vmin=0, vmax=2)
sm = plt.cm.ScalarMappable(cmap=cmap,norm=norm)
sm.set_array([])
cbar = fig.colorbar(sm,ax)
for t in cbar.ax.get_yticklabels():
    t.set_fontsize(15)
    
plt.title("TF expression\n", fontsize=20)
plt.grid(False)
plt.savefig(f"scenic_postpr_heatmaps/Dotheatmap_MDD_F_disease_TFsbyAUC_TFexp_Legend.png", bbox_inches="tight")
plt.savefig(f"scenic_postpr_heatmaps/Dotheatmap_MDD_F_disease_TFsbyAUC_TFexp_Legend.svg", bbox_inches="tight", format="svg")
plt.savefig(f"scenic_postpr_heatmaps/Dotheatmap_MDD_F_disease_TFsbyAUC_TFexp_Legend.pdf", bbox_inches="tight", format="pdf")
plt.show();

In [ ]:
numbers = [1, 2, 3, 4]
val = np.round( np.quantile(rss_long["RSS rank"].values, [.25, .5, .75, 1]), 0)
sizes= val /3

# Create a figure without any content
fig = plt.figure(figsize=(1, 3), dpi=300)

# Create an empty axis
ax = fig.add_subplot(111)

# Plot dots with invisible markers for creating legend
for num, size, v in zip(numbers, sizes, val):
    ax.scatter([], [], s=size, label=f'{v}', color='black')

# Add legend
ax.legend(title='RSS rank\n', loc='center')

# Hide axis
ax.axis('off')
plt.savefig(f"scenic_postpr_heatmaps/Dotheatmap_MDD_F_disease_TFsbyAUC_RSSrank_Legend.png", bbox_inches="tight")
plt.savefig(f"scenic_postpr_heatmaps/Dotheatmap_MDD_F_disease_TFsbyAUC_RSSrank_Legend.svg", bbox_inches="tight", format="svg")
plt.savefig(f"scenic_postpr_heatmaps/Dotheatmap_MDD_F_disease_TFsbyAUC_RSSrank_Legend.pdf", bbox_inches="tight", format="pdf")
plt.show();

Control

In [ ]:
# Calculate RSS

rss_mdd_ctrl = regulon_specificity_scores(auc_mtx=auc_mdd_ctrl, cell_type_series=metadata_mdd.loc[metadata_mdd["Diagnosis"] == "Control", "Celltype"])
rss_mdd_ctrl

In [ ]:
for ct in rss_mdd_ctrl.index :
    rss_mdd_ctrl.loc[ct] =  np.argsort(np.argsort(rss_mdd_ctrl.loc[ct]))

rss_mdd_ctrl

In [ ]:
rss_mdd_ctrl.max(axis=1)
rss_mdd_ctrl.min(axis=1)

In [ ]:
rss_mdd_ctrl["celltype"] = rss_mdd_ctrl.index
rss_mdd_ctrl

In [ ]:
# Oder celltypes

rss_mdd_ctrl.sort_index(inplace=True)
rss_mdd_ctrl

In [ ]:
rss_long = pd.melt(rss_mdd_ctrl, id_vars="celltype")
rss_long.columns = ["celltype", "Regulon", "RSS rank"]
rss_long

In [ ]:
rss_long["Activity"] = 0
rss_long

In [110]:
for reg in auc_mdd_ctrl.columns : 

    for cs in rss_long.celltype.unique() : 

        mean_act = np.mean(auc_mdd_ctrl.loc[metadata_mdd.loc[metadata_mdd["Diagnosis"] == "Control","Celltype"] == cs,reg])
        rss_long.loc[(rss_long["Regulon"] == reg) & (rss_long["celltype"] == cs), "Activity"] = mean_act

In [ ]:
rss_long

In [ ]:
rss_long["Counts"] = 0
rss_long

In [113]:
for reg in rss_long.Regulon.unique() : 
    tf_name = reg.split("_")[0]
    for cs in rss_long.celltype.unique() : 
        if tf_name in genes :
            avg_exp = np.mean(exp_mdd.loc[ metadata_mdd.loc[(metadata_mdd["Diagnosis"] == "Control") & (metadata_mdd["Celltype"] == cs)].index ][tf_name])
        else :
            
            # quick "fix" but check with Hanne
            avg_exp = 0
        rss_long.loc[(rss_long["Regulon"] == reg) & (rss_long["celltype"] == cs), "Counts"] = avg_exp

In [ ]:
rss_long

In [115]:
# Select 10 most active TFs in each cell state

auc_top10 = []
for cs in rss_long.celltype.unique() : 
    tmp = rss_long[rss_long["celltype"] == cs].sort_values(by="Activity", ascending=False)
    auc_top10.extend(list(tmp["Regulon"][:10].values))

In [ ]:
len(auc_top10)
len(set(auc_top10))

In [ ]:
# Make list unique while preserving order
auc_top10 = list(dict.fromkeys(auc_top10))
len(auc_top10)

In [ ]:
rss_long= rss_long.loc[rss_long["Regulon"].isin(auc_top10),:].copy()
rss_long

In [119]:
# Get unique categories for both axes
unique_tf = rss_long['Regulon'].unique()
unique_cs = rss_long['celltype'].unique()

# Calculate the center positions of each tile
tile_centers = [(i, j) for i in range(len(unique_tf)) for j in range(len(unique_cs))]

In [ ]:
pivot_df = rss_long.pivot(index='celltype', columns='Regulon', values='Activity')
pivot_df

In [ ]:
pivot_df = pivot_df[auc_top10]
pivot_df

In [ ]:
pivot_df.columns = [reg.split("_")[0] for reg in pivot_df.columns]
pivot_df.head()

In [128]:
# Cap expression at 2

rss_long.loc[rss_long["Counts"] > 2, "Counts"] = 2

In [ ]:
import matplotlib as mpl

plt.figure(figsize=(15, 3), dpi=300)
sns.heatmap(pivot_df, cmap='magma_r')

cmap = mpl.colormaps.get_cmap('cividis')
for i, (tf, cs) in enumerate(tile_centers):
    size = rss_long.loc[(rss_long['Regulon'] == auc_top10[tf]) & (rss_long['celltype'] == unique_cs[cs]),'RSS rank'].values[0]
    color_value = rss_long.loc[(rss_long['Regulon'] == auc_top10[tf]) & (rss_long['celltype'] == unique_cs[cs]),'Counts'].values[0]

    color = cmap(color_value)
    plt.scatter(tf + 0.5, cs + 0.5, s=size/3 , color=color)

plt.xlabel('TF')
plt.xticks([i+.5 for i in range(pivot_df.shape[1])], list(pivot_df.columns), fontsize=12, rotation=90)
plt.ylabel('Celltype')
plt.yticks([i+.5 for i in range(pivot_df.shape[0])], list(pivot_df.index), fontsize=12)
plt.grid(False)
plt.savefig(f"scenic_postpr_heatmaps/Dotheatmap_MDD_F_control_TFsbyAUC.png", bbox_inches="tight")
plt.savefig(f"scenic_postpr_heatmaps/Dotheatmap_MDD_F_control_TFsbyAUC.svg", bbox_inches="tight", format="svg")
plt.savefig(f"scenic_postpr_heatmaps/Dotheatmap_MDD_F_control_TFsbyAUC.pdf", bbox_inches="tight", format="pdf")
plt.show();

In [ ]:
fig,ax = plt.subplots(figsize=(1,3), dpi=300)

#adding colorbar
norm = mpl.colors.Normalize(vmin=0, vmax=2)
sm = plt.cm.ScalarMappable(cmap=cmap,norm=norm)
sm.set_array([])
cbar = fig.colorbar(sm,ax)
for t in cbar.ax.get_yticklabels():
    t.set_fontsize(15)
    
plt.title("TF expression\n", fontsize=20)
plt.grid(False)
plt.savefig(f"scenic_postpr_heatmaps/Dotheatmap_MDD_F_control_TFsbyAUC_TFexp_Legend.png", bbox_inches="tight")
plt.savefig(f"scenic_postpr_heatmaps/Dotheatmap_MDD_F_control_TFsbyAUC_TFexp_Legend.svg", bbox_inches="tight", format="svg")
plt.savefig(f"scenic_postpr_heatmaps/Dotheatmap_MDD_F_control_TFsbyAUC_TFexp_Legend.pdf", bbox_inches="tight", format="pdf")
plt.show();

In [ ]:
numbers = [1, 2, 3, 4]
val = np.round( np.quantile(rss_long["RSS rank"].values, [.25, .5, .75, 1]), 0)
sizes= val /3

# Create a figure without any content
fig = plt.figure(figsize=(1, 3), dpi=300)

# Create an empty axis
ax = fig.add_subplot(111)

# Plot dots with invisible markers for creating legend
for num, size, v in zip(numbers, sizes, val):
    ax.scatter([], [], s=size, label=f'{v}', color='black')

# Add legend
ax.legend(title='RSS rank\n', loc='center')

# Hide axis
ax.axis('off')
plt.savefig(f"scenic_postpr_heatmaps/Dotheatmap_MDD_F_control_TFsbyAUC_RSSrank_Legend.png", bbox_inches="tight")
plt.savefig(f"scenic_postpr_heatmaps/Dotheatmap_MDD_F_control_TFsbyAUC_RSSrank_Legend.svg", bbox_inches="tight", format="svg")
plt.savefig(f"scenic_postpr_heatmaps/Dotheatmap_MDD_F_control_TFsbyAUC_RSSrank_Legend.pdf", bbox_inches="tight", format="pdf")
plt.show();